In [1]:
import os
import json
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("jhgan/ko-sroberta-multitask")
# snunlp/KR-SBERT-V40K-klueNLI-augSTS
#jhgan/ko-sbert-sts
# sentence-transformers/LaBSE

SEED_NUM = 1
# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

def mixed_search(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    body = {
        "query": {
            "match": {
                "content": {
                    "query": query_str,
                    "boost": 0.002
                }
            }
        },
        "knn": {
            "field": "embeddings",
            "query_vector": query_embedding.tolist(),
            "k": 5,
            "num_candidates": 50,
            "boost": 1
        },
        "size": size
    }

    return es.search(index="test", body=body)


In [3]:
es_username = "엘라스틱_ID값 주기"
es_password = "엘라스틱_패스워드값 주기"

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="./elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                "stoptags": ["E", "J", "SC", "SE", "SF", "SSC", "SSO", "SY", "VCN", "VCP", "VX"]
            }
        }
    },
    "index": {
        "similarity": {
            "lm_jelinek_mercer": {
                "type": "LMJelinekMercer",
                "lambda": 0.7
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {"type": "text", "analyzer": "nori", "similarity": "lm_jelinek_mercer"},
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm" # cosine
        }
    }
}


{'name': 'instance-8205', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'cIGtsxjXSd23Z3fu3H0d-A', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [4]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)
                
# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


In [8]:
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

# mixed_search 사용한 검색 예제
search_result_retrieve = mixed_search(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])


score: 18.95909 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 12.046148 source: 메릴랜드 Space Grant Observatory는 볼티모어에 위치해 있습니다. 학생들은 이 망원경을 방문하여 별, 행성, 달에 대해 배웠습니다. 그들은 아래와 같은 정보를 기록했습니다. 첫째, 별 패턴은 그대로 유지되지만, 하늘에서의 위치는 변하는 것처럼 보입니다. 둘째, 태양, 행성, 달은 하늘에서 움직이는 것처럼 보입니다. 셋째, 켄타우루스자리의 프록시마 성은 우리 태양계에서 가장 가까운 별입니다. 넷째, 북극성은 소 북두칠성이라 불리는 별 패턴의 일부입니다. 그렇다면 태양이 매일 하늘을 가로질러 움직이는 것처럼 보이는 이유는 무엇일까요? 지구가 자전축을 중심으로 회전하기 때문입니다.
score: 10.410804 source: 당신은 금성에 살고 있고, 당신의 망원경이 금성의 두꺼운 구름을 볼 수 있다고 가정한다면, 지구의 달을 관찰할 수 있습니다. 지구의 달은 월식과 만월을 거치며 다양한 단계를 보여줍니다. 여기서는 월식이 아닌 단계를 중심으로 설명하겠습니다.

첫 번째로, 상현 단계를 볼 수 있습니다. 이 단계에서는 달이 점점 커지면서 반달 모양을 보여줍니다. 달의 오른쪽 절반은 밝고, 왼쪽 절반은 어둡습니다. 이는 달이 점점 지구와 태양 사이의 각도가 커지면서 발생하는 현상입니다.

두 번째로, 망 단계를 볼 수 있습니다. 이 단계에서는 달이 지구와 태양 사이에 정확히 위치하여 완전한 원형을 보여줍니다. 

/tmp/ipykernel_1397627/302304740.py:92: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


In [9]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "오픈AI의 API값 넣어주기"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
# persona_function_calling = """
# ## Role: 과학 상식 전문가

# ## Instruction
# - 만약, 과학과 관련없는 메시지에는 적절한 대답을 생성한다.
#   예를들어 merge sort, 심해, 침전, Python, 근친간, 오토마톤, 생태계, 통학 버스, 사람, 피임, 헬륨이, 문맹, 자기장, 노폐물, 글리코겐, 빗방울, 기름, 차량에 대한 메시지에는 적절한 대답을 생성한다.
# - 반드시, 사용자가 대화를 통해 과학에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
# - 질문이 없다면, 적절한 대답을 생성한다.
# """
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 지식에 관해 질문하는 경우에는 반드시 search 함수를 호출한다.
- 나머지 메시지에는 함수 호출 없이 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "User's question in Korean. Full message if the user message is single-turn."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=SEED_NUM,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = mixed_search(standalone_query, 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=SEED_NUM,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response




In [10]:

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../data/edit_eval.jsonl", "sample_submission9.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/tmp/ipykernel_1397627/302304740.py:92: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


Answer: 나무의 분류를 조사하기 위한 방법은 다양합니다. 일반적으로는 잎의 형태, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 분류합니다. 또한, 나무의 용도나 생태학적 특성에 따라서도 분류할 수 있습니다. 하지만 나무를 분류할 때에는 전문적인 지식과 경험이 필요하며, 식물학적 특성과 생태학적 특징을 고려해야 합니다. 따라서 나무의 분류를 위해서는 전문적인 교육과 지식이 필요합니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 죄송합니다, 제가 현재 각 나라의 공교육 지출 현황에 대한 정확한 정보를 제공할 수 없습니다. 더 많은 정보가 필요하시다면 해당 국가의 교육부나 통계청 등의 공식 기관에서 확인하시는 것이 좋습니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 다양한 원인에 의해 발생할 수 있습니다. 일반적으로 기억 상실증은 뇌의 기능이나 구조적인 변화로 인해 발생할 수 있습니다. 이러한 변화는 다양한 요인에 의해 발생할 수 있으며, 스트레스, 외상, 뇌졸중, 알츠하이머 병 등이 그 예시입니다. 그러나 각각의 경우에 따라 원인과 증상이 다를 수 있으므로 정확한 진단과 치료가 필요합니다. 더 자세한 정보를 원하신다면 의사와 상담하시는 것이 좋습니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학 버스는 학교 출퇴근 시간에 많은 인원을 한번에 운송할 수 있는 편리한 수단입니다. 학생들의 안전을 보장하고, 학교로 향하는 사람들의 이동을 원활하게 도와줍니다. 또한, 학교 